In [ ]:
import os
import glob
import cv2
from tqdm import tqdm
import pickle

In [ ]:
cap_dict = {}
train_data, test_data, dev_data = {}, {}, {}
with open(r'.\Flickr8k_Data\Flickr8k.token.txt', 'r', encoding='utf-8') as f:
    for x in f:
        k, v = x.split('\t')
        k = k[:-2]
        if k not in cap_dict:
            cap_dict[k] = [v.strip()]
        else:
            cap_dict[k].append(v.strip())

with open(r'.\Flickr8k_Data\Flickr_8k.trainImages.txt', 'r', encoding='utf-8') as f:
    for x in f:
        train_data[x.strip()] = cap_dict[x.strip()]

with open(r'.\Flickr8k_Data\Flickr_8k.testImages.txt', 'r', encoding='utf-8') as f:
    for x in f:
        test_data[x.strip()] = cap_dict[x.strip()] 

with open(r'.\Flickr8k_Data\Flickr_8k.devImages.txt', 'r', encoding='utf-8') as f:
    for x in f:
        dev_data[x.strip()] = cap_dict[x.strip()]

with open(r'.\Flicker8k_captions\train_data.pickle', 'wb') as f:
    pickle.dump(train_data, f)

# Save dev_data
with open(r'.\Flicker8k_captions\dev_data.pickle', 'wb') as f:
    pickle.dump(dev_data, f)

# Save test_data
with open(r'.\Flicker8k_captions\test_data.pickle', 'wb') as f:
    pickle.dump(test_data, f)

In [ ]:
def batch_resize_images_cv2(source_folder, destination_folder, new_size):
    """
    Reads all JPEG images from a source folder using OpenCV, resizes them,
    and saves them to a destination folder.

    Args:
        source_folder (str): The path to the folder with original images.
        destination_folder (str): The path to the folder to save resized images.
        new_size (tuple): A tuple of (width, height) for the new size.
    """
    # Ensure the destination folder exists
    os.makedirs(destination_folder, exist_ok=True)
    
    # Find all jpeg files in the source folder (case-insensitive)
    search_patterns = [
        os.path.join(source_folder, '*.jpg'),
        os.path.join(source_folder, '*.jpeg'),
        os.path.join(source_folder, '*.JPG'),
        os.path.join(source_folder, '*.JPEG')
    ]
    
    image_paths = []
    for pattern in search_patterns:
        image_paths.extend(glob.glob(pattern))

    if not image_paths:
        print(f"No JPEG images found in '{source_folder}'.")
        return

    print(f"Found {len(image_paths)} images. Starting resizing process...")

    # Loop through all found image paths with a progress bar
    for img_path in tqdm(image_paths, desc="Resizing Images"):
        try:
            # Create the full path for the output file
            file_name = os.path.basename(img_path)
            dest_path = os.path.join(destination_folder, file_name)

            # Optional: Skip if the file has already been resized
            if os.path.exists(dest_path):
                continue
            
            # 1. Read the image using OpenCV
            img = cv2.imread(img_path)

            # cv2.imread returns None if the image cannot be read
            if img is not None:
                # 2. Resize the image
                # cv2.INTER_AREA is recommended for shrinking images (downsampling)
                resized_img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

                # 3. Save the resized image with specified JPEG quality
                cv2.imwrite(dest_path, resized_img, [cv2.IMWRITE_JPEG_QUALITY, 90])
            else:
                print(f"Warning: Could not read image {img_path}. Skipping.")

        except Exception as e:
            print


# Define your source and destination folders
SOURCE_DIR = r'C:\Users\Samsung\Documents\OMSCS\Deep Learning\project\Flickr8k_Data\Flicker8k_Dataset'
DEST_DIR = r'C:\Users\Samsung\Documents\OMSCS\Deep Learning\project\Flickr8k_Data\Flicker8k_Dataset_resized'
    
# Define the target size
NEW_SIZE = (224, 224) # (width, height)

batch_resize_images_cv2(SOURCE_DIR, DEST_DIR, NEW_SIZE)